In [72]:
from frust.stepper import Stepper
from pathlib import Path
from tooltoad.chemutils import xyz2mol
import pandas as pd

In [73]:
structures_path = Path("../structures/test_structs")

mols = {}

for f in structures_path.rglob("*.xyz"):
    with open(f, "r") as file:
        xyz_block = file.read()
        mol = xyz2mol(xyz_block)
        mols[f.stem] = (mol, [0])

In [74]:
mols

{'ts2_guess': (<rdkit.Chem.rdchem.Mol at 0x1744a9d20>, [0]),
 'TS2(min-gode)': (<rdkit.Chem.rdchem.Mol at 0x1743b9d20>, [0]),
 'TS2(font-2)': (<rdkit.Chem.rdchem.Mol at 0x1744a92a0>, [0]),
 'TS2(min-bad)': (<rdkit.Chem.rdchem.Mol at 0x1743bb8b0>, [0]),
 'ts1_guess': (<rdkit.Chem.rdchem.Mol at 0x1743baff0>, [0])}

In [75]:
list(mols.keys())

['ts2_guess', 'TS2(min-gode)', 'TS2(font-2)', 'TS2(min-bad)', 'ts1_guess']

In [76]:
step = Stepper(list(mols.keys()), step_type="IRC", save_output_dir=False)

In [77]:
df0 = step.build_initial_df(mols)

In [78]:
df0

,custom_name,ligand_name,rpos,constraint_atoms,cid,smiles,atoms,coords_embedded,energy_uff
0,ts2_guess,guess,<NA>,None,0,None,"[C, C, C, C, C, C, H, H, H, H, N, C, C, C, C, ...","[(-2.270329, -1.347202, 0.849568), (-1.131801,...",None
1,TS2(min-gode),TS2(min-gode),<NA>,None,0,None,"[C, C, C, C, C, C, H, H, H, H, N, C, C, C, C, ...","[(2.215338, 0.845533, 0.885553), (1.422874, -0...",None
2,TS2(font-2),TS2(font-2),<NA>,None,0,None,"[C, C, C, C, C, C, H, H, H, H, B, H, C, C, C, ...","[(-2.121331, -1.926482, 1.302524), (-0.776694,...",None
3,TS2(min-bad),TS2(min-bad),<NA>,None,0,None,"[C, C, C, C, C, C, H, H, H, H, N, C, C, C, C, ...","[(-0.544828, 1.901314, 1.75786), (-0.741228, 0...",None
4,ts1_guess,guess,<NA>,None,0,None,"[C, C, C, C, C, C, H, H, H, H, H, B, N, C, C, ...","[(-1.318292, -2.055119, 0.53709), (-0.023698, ...",None


In [ ]:
dfs = []
for i in range(len(df0)):
    df_tmp = df0.iloc[[i]]
    dfs.append(df_tmp)

def run_irc(df, debug=False):
    name = df["custom_name"].iloc[0]

    step = Stepper([name], step_type="IRC", debug=debug, save_output_dir=False)

    detailed_inp = """%geom
    Calc_Hess true
    end
    """
    options = {
        "wB97X-D3": None,
        "6-31G**" : None,
        "TightSCF": None,
        "NoSym"    : None,
        "IRC"      : None,
    }

    df1 = step.orca(df, "DFT-IRC", options, detailed_inp, save_step=True)

    df1.to_parquet(f"df_irc_{name}.parquet")

In [89]:
for dfi in dfs:
    run_irc(dfi, True)

2025-06-25 13:42:58 INFO  frust.stepper: [DFT-IRC-wB97X-D3-6-31G**-NoSym] row 0 (ts2_guess)…
2025-06-25 13:42:58 INFO  frust.stepper: [DFT-IRC-wB97X-D3-6-31G**-NoSym] row 1 (TS2(min-gode))…
2025-06-25 13:42:58 INFO  frust.stepper: [DFT-IRC-wB97X-D3-6-31G**-NoSym] row 2 (TS2(font-2))…
2025-06-25 13:42:58 INFO  frust.stepper: [DFT-IRC-wB97X-D3-6-31G**-NoSym] row 3 (TS2(min-bad))…
2025-06-25 13:42:58 INFO  frust.stepper: [DFT-IRC-wB97X-D3-6-31G**-NoSym] row 4 (ts1_guess)…
